In [3]:
from flash_attn import flash_attn_with_kvcache
import torch
from torch.nn import functional as F

In [72]:
lengths = torch.tensor([0, 0, 0]).to(torch.int32).cuda()

q = torch.rand(3, 5, 32, 128).to(torch.bfloat16).cuda()
k = torch.rand(3, 5, 32, 128).to(torch.bfloat16).cuda()
v = torch.rand(3, 5, 32, 128).to(torch.bfloat16).cuda()

ks, vs = [], []
for length in lengths:
    ks.append(torch.rand(1, length, 32, 128).to(torch.bfloat16).cuda())
    vs.append(torch.rand(1, length, 32, 128).to(torch.bfloat16).cuda())

k_cache = torch.cat([F.pad(ks[i], (0, 0, 0, 0, 0, 30 - lengths[i])) for i in range(3)])
v_cache = torch.cat([F.pad(vs[i], (0, 0, 0, 0, 0, 30 - lengths[i])) for i in range(3)])

q = q.to(torch.bfloat16).cuda()
k_cache = k_cache.to(torch.bfloat16).cuda()
v_cache = v_cache.to(torch.bfloat16).cuda()
q.shape, k_cache.shape, v_cache.shape

(torch.Size([3, 5, 32, 128]),
 torch.Size([3, 30, 32, 128]),
 torch.Size([3, 30, 32, 128]))

In [73]:
k_cache = torch.zeros(3, 100, 32, 128).to(torch.bfloat16).cuda()
v_cache = torch.zeros(3, 100, 32, 128).to(torch.bfloat16).cuda()

In [74]:
separate = []
for i in range(3):
    s = slice(i, i+1)
    attnout = flash_attn_with_kvcache(q[s], ks[i], vs[i], causal=True)
    separate.append(attnout)
separate = torch.cat(separate)
separate.shape

torch.Size([3, 5, 32, 128])

In [75]:
together = flash_attn_with_kvcache(q, k_cache, v_cache, k=k, v=v,causal=True, cache_seqlens=lengths)
together.shape

torch.Size([3, 5, 32, 128])

In [76]:
torch.allclose(separate, together, atol=1e-3, rtol=1e-3)

False

In [77]:
torch.where(k_cache[0] != 0)

(tensor([0, 0, 0,  ..., 4, 4, 4], device='cuda:0'),
 tensor([ 0,  0,  0,  ..., 31, 31, 31], device='cuda:0'),
 tensor([  0,   1,   2,  ..., 125, 126, 127], device='cuda:0'))

In [81]:
k_cache[0, 1]

tensor([[6.9141e-01, 2.6562e-01, 3.8672e-01,  ..., 7.7344e-01, 8.8672e-01,
         8.5547e-01],
        [5.1953e-01, 1.1816e-01, 6.1279e-02,  ..., 2.9297e-01, 8.7402e-02,
         3.1641e-01],
        [6.6016e-01, 8.3984e-01, 5.2734e-01,  ..., 4.2773e-01, 5.6641e-01,
         7.8516e-01],
        ...,
        [2.2559e-01, 9.7168e-02, 3.6523e-01,  ..., 9.6484e-01, 9.1016e-01,
         8.1250e-01],
        [3.2812e-01, 1.0000e+00, 9.4922e-01,  ..., 9.1797e-01, 9.4922e-01,
         4.8438e-01],
        [9.3359e-01, 1.2061e-01, 2.3926e-01,  ..., 4.1260e-02, 9.1934e-04,
         5.8594e-01]], device='cuda:0', dtype=torch.bfloat16)

In [63]:
k_cache.shape

torch.Size([3, 100, 32, 128])